<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/title.png' width=500/></center>

<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/practice3.png' width=300/></center>

<center>본 실습에서 요구하는 내용 외에도 다양한 내용을 스스로 추가하여  탐색하고 분석해 보기를 권고합니다.</center>

# 종합실습 - 판매 현황 분석

<img src='https://raw.githubusercontent.com/Jangrae/img/master/retail.png' width="650" align="left">

# 1.환경 설정

**1)라이브러리 불러오기**

- 사용할 라이브러리를 불러오세오.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

**2) 한글 설정**

- Jupyter Notebook 시각화에 한글이 표시되게 설정합니다.

In [ ]:
# 한글 폰트설정 #1
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 한글 폰트설정 #2: 구글 코랩 포함
#!pip install koreanize_matplotlib
#import koreanize_matplotlib

**3) 데이터 읽어오기**

- 다음 경로의 파일을 읽어와 각각의 데이터프레임을 만듭니다.
- 데이터프레임 이름은 customers, sales로 합니다.
- 파일 경로
    - 'https://raw.githubusercontent.com/Jangrae/csv/master/customers.csv'    
    - 'https://raw.githubusercontent.com/Jangrae/csv/master/sales.csv'

In [ ]:
# customers 데이터 읽어오기
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/customers.csv'
customers = pd.read_csv(path)

# 확인
customers.head()

In [ ]:
# sales 데이터 읽어오기
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/sales.csv'
sales = pd.read_csv(path)

# 확인
sales.head()

# 2.데이터 전처리 #1

**1) 고객별 집계**

- sales 데이터프레임에서 고객(CustomerID)별 주문 수량(Qty) 합, 금액(Amt) 합을 집계하여 cust_sales 데이터프레임을 선언합니다.

In [ ]:
# 고객별 집계
cust_sales = sales.groupby(by='CustomerID', as_index=False)[['Qty', 'Amt']].sum()

# 확인
cust_sales.head()

**2) 데이터 병합**

- customers, cust_sales 데이터프레임을 **CustomerID** 열을 기준으로 **outer** 방식으로 **병합(=조인)** 하여 **cst** 데이터프레임으로 선언합니다.

- 이제부터 모든 작업은 cst 데이터프레임을 대상으로 합니다.

In [ ]:
# 병합
cst = pd.merge(customers, cust_sales, on='CustomerID', how='outer')

# 확인
cst.head()

**3) 결측치 확인**

- 결측치가 있는지 확인합니다.

In [ ]:
# 결측치 확인
cst.isna().sum()

**4) 결측치 제거**

- 결측치가 있는 행은 제거합니다.

In [ ]:
# 결측치 제거
cst.dropna(inplace=True)

# 확인
cst.isna().sum()

**5) 열 추가: RegisterYear, RegisterMonth**

- 다음 구문을 실행해서 RegisterYear, RegisterMonth 열을 추가합니다.

In [ ]:
# RegisterYear, RegisterMonth 열 추가
cst['RegisterDate'] = pd.to_datetime(cst['RegisterDate'])
cst['RegisterYear'] = cst['RegisterDate'].dt.year
cst['RegisterMonth'] = cst['RegisterDate'].dt.month

# 확인
cst.head()

**6) 열 추가: Age**

- 'Age = 2017 - 출생연도(BirthYear)'  공식을 이용해 Age 열을 추가합니다.

In [ ]:
# Age 추가
cst['Age'] = 2017 - cst['BirthYear']

# 확인
cst.head()

**7) 열 추가: Age2**

- 'Age2 = Age // 10 * 10' 공식을 이용해 Age2 열을 추가합니다.

In [ ]:
# Age2 추가
cst['Age2'] = cst['Age'] // 10 * 10

# 확인
cst.head()

**8) 열 추가: Level**

- 다음 Amt 열의 값 기준에 따라 등급을 갖는 Level 열을 추가합니다.
    * ~ 30,000: Family
    * ~ 100,000: Bronze
    * ~ 300,000: Silver
    * ~ 600,000: Gold
    * 600,000 ~ : Platinum

In [ ]:
# Level 열 추가
bin = [-np.inf, 30000, 100000, 300000, 600000, np.inf]
label = ['Family', 'Bronze', 'Silver', 'Gold', 'Platinum']
cst['Level'] = pd.cut(cst['Amt'], bins=bin, labels=label)

# 확인
cst.head()

**9) 불필요한 열 제거**

- RegisterDate, Address, BirthYear, Addr2 열을 제거합니다.

In [ ]:
# 불필요한 열 제거
cols = ['RegisterDate', 'Address', 'BirthYear', 'Addr2']
cst.drop(cols, axis=1, inplace=True)

# 확인
cst.head()

# 3.데이터 집계

**1) 지역별 고객 수**

- 지역(Addr1)별 고객(CustomerID) 수를 집계하여 tmp 데이터프레임으로 선언합니다.
- tmp 데이터프레임 열 이름을 '지역', '고객수'로 변경합니다.
- tmp 데이터프레임을 고객수를 기준으로 내림차순 정렬합니다.
- tmp 데이터프레임의 상위 5개 행만을 추출해 tmp_5 데이터프레임으로 선언합니다.

In [ ]:
# 지역별 고객 수
tmp = cst.groupby(by='Addr1', as_index=False)['CustomerID'].count()
tmp.columns = ['지역', '고객수']
tmp.sort_values(by='고객수', ascending=False, inplace=True)
tmp_5 = tmp.head(5)

# 확인
tmp

- tmp_5 데이터프레임을 막대 그래프로 시각화 합니다.

In [ ]:
# 지역별 고객 수 TOP 5 시각화
plt.bar(tmp_5['지역'], tmp_5['고객수'])
plt.xlabel('지역')
plt.ylabel('고객수')
plt.show()

**2) 연령대별 고객 수**

- 연령대(Age2)별 고객 수를 집계하여 tmp 데이터프레임으로 선언합니다.
- tmp 데이터프레임 열 이름을 '연령대', '고객수'로 변경합니다.
- tmp 데이터프레임 연령대 열의 데이터 형식을 문자열로 변환합니다.

In [ ]:
# 연령대별 고객 수
tmp = cst.groupby(by='Age2', as_index=False)['CustomerID'].count()
tmp.columns = ['연령대', '고객수']
tmp['연령대'] = tmp['연령대'].astype(str)

# 확인
tmp

- tmp 데이터프레임을 막대 그래프로 시각화 합니다.

In [ ]:
# 연령대별 고객 수 시각화
plt.bar(tmp['연령대'], tmp['고객수'])
plt.xlabel('연령대')
plt.ylabel('고객수')
plt.show()

**3) 등급별 판매 금액**

- 등급(Level)별 판매 금액(Amt)을 집계하여 tmp 데이터프레임으로 선언합니다.
- tmp 데이터프레임 열 이름을 '등급', '합계금액'으로 변경합니다.
- 합계금액은 10,000 단위로 변경합니다.

In [ ]:
# 등급별 판매 금액
tmp = cst.groupby(by='Level', as_index=False)[['Amt']].sum()
tmp.columns = ['등급', '합계금액']
tmp['합계금액'] = tmp['합계금액'] / 10000

# 확인
tmp

- tmp 데이터프레임을 막대 그래프로 시각화 합니다.

In [ ]:
# 등급별 판매 금액 시각화
plt.bar(tmp['등급'], tmp['합계금액'])
plt.xlabel('등급')
plt.ylabel('합계금액(천)')
plt.show()

# 4.전처리 #2

**1) 열 제거**

- Addr1, RegisterYear, RegisterMonth, Age 열을 제거합니다.

In [ ]:
# 열 제거
cst.drop(['Addr1', 'RegisterYear', 'RegisterMonth', 'Age'], axis=1, inplace=True)

# 확인
cst.head()

**2) 열 이름 변경**

- Age2 열 이름을 Age로 변경합니다.

In [ ]:
# 열 이름 변경: Age2 --> Age
cst.rename(columns={'Age2': 'Age'}, inplace=True)

# 확인
cst.head()

**3) 가변수화**

- Gender, Age, Level 열을 가변수화 합니다.

In [ ]:
# 가변수화
cst = pd.get_dummies(cst, columns=['Gender', 'Age', 'Level'], drop_first=True)

# 확인
cst.head()